# Temperature Ramps

In [1]:
#-----Import esential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Declare some important variables

t_0 = 5.4;
t_1 = 5.6;
t_2 = 5.8;
t_3 = 6.0;
p_max  = 1.4;
steps_ramp = 5e4;


#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM_5.4_P_1.4_ramp.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode='shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

#----Integrate using NPT

temp1 = hoomd.variant.linear_interp(points = [(0,t_0), (steps_ramp, t_1)]);
temp2 = hoomd.variant.linear_interp(points = [(0,t_1), (steps_ramp, t_2)]);
temp3 = hoomd.variant.linear_interp(points = [(0,t_2), (steps_ramp, t_3)]);

npt = hoomd.md.integrate.npt(group = mesogens, kT = temp1, tau = 1.0, tauP = 1.0, P = p_max);
npt.randomize_velocities(seed = 42)

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [2]:
#-----Ramp1 T : 5.4 -> 5.6
#-----Write output and Run the Simulation

log_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.log"
gsd_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True);

In [3]:
hoomd.run(steps_ramp)
system.box.get_volume()

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 1352181 / 1402000 | TPS 18.0959 | ETA 00:45:53
Time 00:00:20 | Step 1352374 / 1402000 | TPS 19.24 | ETA 00:42:59
Time 00:00:30 | Step 1352574 / 1402000 | TPS 19.8317 | ETA 00:41:32
Time 00:00:40 | Step 1352763 / 1402000 | TPS 18.8794 | ETA 00:43:27
Time 00:00:50 | Step 1352957 / 1402000 | TPS 19.3677 | ETA 00:42:12
Time 00:01:00 | Step 1353154 / 1402000 | TPS 19.6915 | ETA 00:41:20
Time 00:01:10 | Step 1353352 / 1402000 | TPS 19.6944 | ETA 00:41:10
Time 00:01:20 | Step 1353554 / 1402000 | TPS 20.1696 | ETA 00:40:01
Time 00:01:30 | Step 1353752 / 1402000 | TPS 19.7915 | ETA 00:40:37
Time 00:01:40 | Step 1353942 / 1402000 | TPS 18.9217 | ETA 00:42:19
Time 00:01:50 | Step 1354127 / 1402000 | TPS 18.4811 | ETA 00:43:10
Time 00:02:

14386.339350597402

In [4]:
system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x1037538d0>),
             ('particles', <hoomd.data.particle_data at 0x111930278>),
             ('number_density', 0.6255934731322926),
             ('bonds', <hoomd.data.bond_data at 0x1119302e8>),
             ('angles', <hoomd.data.angle_data at 0x111930358>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x1119303c8>),
             ('impropers', <hoomd.data.dihedral_data at 0x111930438>),
             ('constraints', <hoomd.data.constraint_data at 0x1119304a8>),
             ('pairs', <hoomd.data.bond_data at 0x111930518>),
             ('timestep', 1402000)])

In [6]:
#-----Ramp2 T : 5.6 -> 5.8
#-----Update parameters

npt.set_params(kT = temp2)

#-----Write output and Run the Simulation

log_file = "T_" + str(t_2) + "_P_" + str(p_max) + "_ramp.log"
gsd_file = "T_" + str(t_2) + "_P_" + str(p_max) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_2) + "_P_" + str(p_max) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True);

In [ ]:
hoomd.run(steps_ramp)
system.box.get_volume()

** starting run **
Time 00:46:39 | Step 1402184 / 1452000 | TPS 18.3839 | ETA 00:45:09
Time 00:46:49 | Step 1402383 / 1452000 | TPS 19.8288 | ETA 00:41:42
Time 00:46:59 | Step 1402575 / 1452000 | TPS 19.1012 | ETA 00:43:07
Time 00:47:09 | Step 1402763 / 1452000 | TPS 18.7771 | ETA 00:43:42
Time 00:47:19 | Step 1402945 / 1452000 | TPS 18.1556 | ETA 00:45:01
Time 00:47:29 | Step 1403136 / 1452000 | TPS 18.9455 | ETA 00:42:59
Time 00:47:40 | Step 1403329 / 1452000 | TPS 19.2756 | ETA 00:42:05
Time 00:47:50 | Step 1403526 / 1452000 | TPS 19.6849 | ETA 00:41:02
Time 00:48:00 | Step 1403713 / 1452000 | TPS 18.683 | ETA 00:43:04
Time 00:48:10 | Step 1403899 / 1452000 | TPS 18.553 | ETA 00:43:12
Time 00:48:20 | Step 1404083 / 1452000 | TPS 18.3011 | ETA 00:43:38
Time 00:48:30 | Step 1404279 / 1452000 | TPS 19.5351 | ETA 00:40:42
Time 00:48:40 | Step 1404479 / 1452000 | TPS 19.9827 | ETA 00:39:38
Time 00:48:50 | Step 1404651 / 1452000 | TPS 17.0562 | ETA 00:46:16
Time 00:49:00 | Step 1404820 / 

In [ ]:
system.get_metadata()

In [ ]:
#-----Ramp2 T : 5.8 -> 6.0
#-----Update parameters

npt.set_params(kT = temp3)

#-----Write output and Run the Simulation

log_file = "T_" + str(t_3) + "_P_" + str(p_max) + "_ramp.log"
gsd_file = "T_" + str(t_3) + "_P_" + str(p_max) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_3) + "_P_" + str(p_max) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True);

In [ ]:
hoomd.run(steps_ramp)
system.box.get_volume()

In [ ]:
system.get_metadata()